# Example use of libgcode

This file contains some programs I use on my CNC.
For instance, surfacing or making a small workholing jig.
I use this when writting the gcode is faster than doing the full CAD+CAM.

## Setup

All the imports.
First, adding the additional repo for the dependecies and then loading `libgcode`.
Don't forget to run `sbt publishLocal` in the `libgcode` folder.

In [1]:
import coursierapi._
interp.repositories() ++= Seq(MavenRepository.of("https://github.com/dzufferey/my_mvn_repo/raw/master/repository"))

import coursierapi._


In [2]:
import $ivy.`io.github.dzufferey::libgcode:0.1-SNAPSHOT`
import libgcode._
import libgcode.extractor._
import java.io._
import java.nio.file._

Checking https://github.com/dzufferey/my_mvn_repo/raw/master/repository/io/github/dzufferey/misc-scala-utils_2.13/0.1-SNAPSHOT/misc-scala-utils_2.13-0.1-SNAPSHOT.pom
Checked https://github.com/dzufferey/my_mvn_repo/raw/master/repository/io/github/dzufferey/misc-scala-utils_2.13/0.1-SNAPSHOT/misc-scala-utils_2.13-0.1-SNAPSHOT.pom
Downloaded https://github.com/dzufferey/my_mvn_repo/raw/master/repository/io/github/dzufferey/misc-scala-utils_2.13/0.1-SNAPSHOT/misc-scala-utils_2.13-0.1-SNAPSHOT.pom
Checking https://github.com/dzufferey/my_mvn_repo/raw/master/repository/io/github/dzufferey/misc-scala-utils_2.13/0.1-SNAPSHOT/misc-scala-utils_2.13-0.1-SNAPSHOT.pom.sha1
Checked https://github.com/dzufferey/my_mvn_repo/raw/master/repository/io/github/dzufferey/misc-scala-utils_2.13/0.1-SNAPSHOT/misc-scala-utils_2.13-0.1-SNAPSHOT.pom.sha1
Downloaded https://github.com/dzufferey/my_mvn_repo/raw/master/repository/io/github/dzufferey/misc-scala-utils_2.13/0.1-SNAPSHOT/misc-scala-utils_2.13-0.1-SNAPS

import $ivy.$                                           

import libgcode._

import libgcode.extractor._

import java.io._

import java.nio.file._

We are now ready to go!

## Shared Elements

Let's pretend to do "good software engineering" and factor common parts rather than copy-pasting stuff all over the place. :)

In [6]:
val absolute = G(90)
val incremental = G(91)
val mm = G(21)

abstract class Program {

    //start and finish Z
    val clearanceZ = 5.0
    // default feed
    val feed = 100
    
    
    def header = Seq(
        mm,
        absolute,
        Empty(F(feed)),
        G(0, Z(clearanceZ))
    )
    
    def body: Seq[Command] //Here you do what you need to do!!
    
    def footer = Seq(
        G(0, Z(clearanceZ)),
        M(2)
    )
    
    def program = header ++ body ++ footer

    def save(fileName: String) = {
        Printer(program, fileName)
    }
    
    override def toString = {
        val writer = new StringWriter
        Printer(program, new BufferedWriter(writer))
        writer.toString
    }
    
}

absolute: Command = Command(G, List(90), ArraySeq(), None, None)
incremental: Command = Command(G, List(91), ArraySeq(), None, None)
mm: Command = Command(G, List(21), ArraySeq(), None, None)
defined class Program

## Surfacing

Some simple programs to make flat surfaces.

In [7]:
class CicularSurface(cutterDiameter: Double, maxRadius: Double,
                     stepOver: Double = 0.9, precision: Double = 0.5,
                     climb: Boolean = true) extends Program {

    override val feed = 200
    val plungeFeed: Double = 100
    
    val effectiveCutting = cutterDiameter * stepOver
    val maxCuttingRadius = maxRadius - cutterDiameter / 2
    def r(t: Double) = effectiveCutting * (1 + t /2 / math.Pi)
    def x(t: Double) = r(t) * math.cos(if (climb) t else -t)
    def y(t: Double) = r(t) * math.sin(if (climb) t else -t)
    //find the next t with for the given precision
    def nextStep(t: Double) = precision / r(t)
    
    def body = {
        val cmds = scala.collection.mutable.ArrayBuffer.empty[Command]
        cmds += G(0, X(effectiveCutting), Y(0.0)) // init position
        cmds += G(1, Z(0.0), F(plungeFeed)) // plunge
        cmds += Empty(F(feed)) // setNormalFeed
        // first do a spiral from the scenter to maxRadius
        var t = nextStep(0.0)
        while (r(t) < maxCuttingRadius) {
            cmds += G(1, X(x(t)), Y(y(t)))
            t += nextStep(t)
        }
        val delta = nextStep(t)
        val oldT = t
        //then do a normal circle at maxRadius
        while (t < oldT + 2 * math.Pi) {
            cmds += G(1,
                      X(maxCuttingRadius * math.cos(if (climb) t else -t)),
                      Y(maxCuttingRadius * math.sin(if (climb) t else -t)))
            t += delta
        }
        cmds.toSeq
    }
}

new CicularSurface(6, 45).save("spiral_test.nc")
new CicularSurface(6, 45, climb = false).save("spiral_test2.nc")

defined class CicularSurface

## Engraving

Test engraving with a V-bit.
That particular one was for a gift.

In [8]:
class EngravingTest extends Program {
    
    override val feed = 100
    
    // generated from a bezier curve using blender
    val points_half = Seq(
       ( 0.000000, 11.086214, 0.000000), 
       ( 0.806945, 11.764009,-0.015200),
       ( 1.640551, 12.316256,-0.059065),
       ( 2.493831, 12.747746,-0.128987),
       ( 3.360688, 13.063274,-0.222361),
       ( 4.235025, 13.267637,-0.336582),
       ( 5.110744, 13.365633,-0.469043),
       ( 5.981749, 13.362055,-0.617139),
       ( 6.841942, 13.261698,-0.778264),
       ( 7.685225, 13.069359,-0.949813),
       ( 8.505503, 12.789831,-1.129178),
       ( 9.296677, 12.427912,-1.313756),
       (10.052650, 11.988398,-1.500939),
       (10.767326, 11.476082,-1.688122),
       (11.434607, 10.895760,-1.872699),
       (12.048395, 10.252230,-2.052064),
       (12.602596,  9.550283,-2.223613),
       (13.091107,  8.794720,-2.384738),
       (13.507836,  7.990332,-2.532834),
       (13.846685,  7.141917,-2.665295),
       (14.101556,  6.254269,-2.779516),
       (14.266351,  5.332184,-2.872890),
       (14.334973,  4.380458,-2.942812),
       (14.301327,  3.403887,-2.986676),
       (14.159315,  2.407268,-3.001877),
       (13.905559,  1.427261,-2.986677),
       (13.547565,  0.495462,-2.942812),
       (13.095561, -0.390940,-2.872890),
       (12.559774, -1.234758,-2.779516),
       (11.950432, -2.038803,-2.665295),
       (11.277760, -2.805887,-2.532834),
       (10.551985, -3.538822,-2.384738),
       ( 9.783336, -4.240419,-2.223613),
       ( 8.982038, -4.913491,-2.052064),
       ( 8.158319, -5.560850,-1.872699),
       ( 7.322405, -6.185307,-1.688122),
       ( 6.484524, -6.789674,-1.500939),
       ( 5.654902, -7.376763,-1.313756),
       ( 4.843767, -7.949385,-1.129178),
       ( 4.061346, -8.510353,-0.949813),
       ( 3.317865, -9.062479,-0.778264),
       ( 2.623551, -9.608573,-0.617139),
       ( 1.988631,-10.151449,-0.469043),
       ( 1.423333,-10.693916,-0.336582),
       ( 0.937883,-11.238791,-0.222361),
       ( 0.542508,-11.788879,-0.128987),
       ( 0.247436,-12.346998,-0.059065),
       ( 0.062892,-12.915956,-0.015201),
       ( 0.000000,-13.498569, 0.000000)
    )

    val scalingX = 2.0
    val scalingY = 2.0
    val scalingZ = 1.0

    val points = {
        val p0 = points_half ++ points_half.reverse.tail.map{ case (x,y,z) => (-x,y,z) }
        p0.map{ case (x,y,z) => (scalingX * x, scalingY * y, scalingZ * z) }
    }

    def layer(zOffset: Double) = {
        points.map{ case (x,y,z) => G(1, X(x), Y(y), Z(z + zOffset)) }
    }
    
    def body = {
        val initPos = G(0, X(points.head._1), Y(points.head._2))
        val zPasses = Seq(2.0, 1.0, 0.5, 0.0, -0.2)
        initPos +: zPasses.flatMap(layer)
    }
   
}

new EngravingTest().save("engraving_test.nc")

defined class EngravingTest

## Grinding

A grinding job.
So it moves very slowly along one axis.

In [11]:
def nbrSteps(start: Double, end: Double, stepSize: Double): Int = ((end - start) / stepSize).abs.floor.toInt
def direction(start: Double, end: Double) = (end - start).sign

val stepSize = 0.01
val yMin = -10.0
val yMax =  10.0

class GrindingZ extends Program {

    val zStart    = 0.0
    val zEnd      = -3.0

    def layer(zIdx: Int) = Seq(
        G(1, Z(zStart + zIdx * stepSize * direction(zStart, zEnd))),
        G(1, Y(yMax)),
        G(0, X(1.0)),
        G(0, Y(yMin)),
        G(0, X(0.0))
    )

    def body = for (z <- 0 to nbrSteps(zStart, zEnd, stepSize); cmd <- layer(z)) yield cmd

}

new GrindingZ().save("grinding_z.nc")

//this time move alone the x axis
class GrindingX extends Program {

    val xStart    = 0.0
    val xEnd      = -3.5

    def layerX(xIdx: Int) = Seq(
        G(1, X(xStart + xIdx * stepSize * direction(xStart, xEnd).sign)),
        G(1, Y(yMax)),
        G(1, Y(yMin))
    )

    val body = {
        val initPos = Seq(
            G(0, X(0), Y(yMin)),
            G(1, Z(0)))
        val layers = for (i <- 0 to nbrSteps(xStart, xEnd, stepSize); cmd <- layerX(i)) yield cmd
        initPos ++ layers
    }

}

new GrindingX().save("grinding_x.nc")

defined function nbrSteps
defined function direction
stepSize: Double = 0.01
yMin: Double = -10.0
yMax: Double = 10.0
defined class GrindingZ
defined class GrindingX

## Misc

In [12]:
object Jig1 {

    val n = 4
    val dy = 15.0
    val toKeep = 5.0
    val slotDepth = 3.5
    val slotWidth = 8.0
    val drillDepth = Z(-10)

    val cutterDiam = 6.0
    val stepOver = 0.9
    val feedMetal = F(120)
    val feedWood = F(200)
    val feedZ = F(100)
    val clearZ = Z(5)
    
    val yMin = Y(-10 - cutterDiam)
    val yMax = Y((n-1)*dy + 10 + cutterDiam)
    def nextY(i: Int) = if (i % 2 == 0) yMax else yMin
    
    protected def buffer = {
        val cmds = scala.collection.mutable.ArrayBuffer.empty[Command]
        cmds += mm
        cmds += absolute
        cmds += G(1, clearZ, feedZ)
        cmds
    }
    
    def slot = {
        val cmds = buffer
        val passes = ( slotWidth / (cutterDiam * stepOver) ).ceil.toInt
        assert(passes > 1 && slotWidth > cutterDiam)
        val effectiveSlot = slotWidth - cutterDiam
        cmds += G(0, X(-effectiveSlot/2), yMin)
        cmds += G(1, Z(-slotDepth), feedZ)    
        for (i <- 0 until passes) {
            cmds += G(1, X(-effectiveSlot/2 + i * effectiveSlot / (passes-1)), feedWood)
            cmds += G(1, nextY(i))
        }
        cmds += G(0, X(0), Y(0))
        cmds += M(2)
        cmds.toSeq
    }
    
    def drill = {
        val cmds = buffer
        cmds += G(0, X(0), Y(0))
        for (i <- 0 until 4) {
            cmds += G(0, Y(i * dy))
            cmds += G(1, drillDepth)
            cmds += G(1, clearZ)
        }
        cmds += G(0, X(0), Y(0))
        cmds += M(2)
        cmds.toSeq
    }
    
    def cut = {
        val dx = (toKeep + cutterDiam) / 2
        val cmds = buffer
        cmds += G(0, X(-dx), yMin)
        cmds += G(1, Z(0), feedZ)
        cmds += G(1, yMax, feedMetal)
        cmds += G(0, X(dx))
        cmds += G(1, yMin)
        cmds += G(1, clearZ, feedZ)
        cmds += M(2)
        cmds.toSeq
    }
    
    def run = {
        Printer(slot, "jig1_slot.nc")
        Printer(drill, "jig1_drill.nc")
        Printer(cut, "jig1_cut.nc")
    }
}

Jig1.run

defined object Jig1